In [1]:
# import necessary libraries
import pandas as pd
import numpy as np
import scipy as sp
from scipy import special

from pylab import *

from pyomo.environ import *
import random

In [ ]:
# read the load profile from excel sheet
path = r"C:\Users\lenovo\Desktop\python_files\School_2016_2017_KW.xlsx"

# create a dataframe and convert to list
columns = [ 0, 1, 2, 3] 

df = pd.read_excel(path, usecols = columns, nrows = 96)

time = df["Date"].tolist()
load = df["Load"].tolist()
cost = df["Cost"].tolist()
solar = df["Solar"].tolist()

In [ ]:
# define optimization variables
# Qm = maximum rate of charge/discharge 1MW
# S = maximum storage capacity 2MWh
# z_t = energy stored in the battery at time stamp t (State of charge)
# define concrete model

model = ConcreteModel()

model.Tmax = len(time) - 1
model.T = RangeSet(0,model.Tmax)

model.Qm = Param(initialize = 1000) # in kW
model.Sm = Param(initialize = 2000) # in kWh
model.Qin = Var(model.T, domain=NonNegativeReals)
model.Qout = Var(model.T, domain=NonNegativeReals)

model.Bc = Var(model.T,domain = Binary, initialize = 0)
model.Bd = Var(model.T,domain = Binary, initialize = 0)

model.z = Var(model.T, within = NonNegativeReals, bounds = (0,model.Sm))
model.qin = Var(model.T, domain=NonNegativeReals)
model.qout = Var(model.T, domain=NonNegativeReals)
eta = 0.92


In [ ]:
# define objective - cost function
model.obj = Objective(expr = sum(cost[t]*(load[t]-model.qout[t]-solar[t]) for t in model.T), sense = minimize)

In [ ]:
# define battery dynamics and maximum charge of the battery constraint, z is the state of charge at time t 
def SoC(model, t):
    if t == model.T.first():
        return model.z[t] == 0.1*model.Sm # starts with 50% charging
    elif solar[t] > load[t]:
        return model.z[t] == model.z[t-1] + 0.25*np.sqrt(eta)*model.qin[t-1] 
    elif solar[t] < load[t]:
        return model.z[t] == model.z[t-1] - 0.25*model.qout[t-1]/np.sqrt(eta)
    else:
        return model.z[t] == model.z[t-1]
model.battery_dynamics = Constraint( model.T, rule=SoC)

def disch_lim(model, t):
    return model.qout[t] <= model.Bd[t]*min(model.Qm, max(0, load[t] - solar[t]))
model.discharge = Constraint(model.T, rule=disch_lim)

def charge_lim(model, t):
    return model.qin[t] <= model.Bc[t]*min(model.Qm, max(0, - load[t] + solar[t]))
model.charge = Constraint(model.T, rule=charge_lim)

def binr(model, t):
    return model.Bc[t] + model.Bd[t] <=1
model.binry = Constraint(model.T, rule = binr)

def demand_positive(model, t):
    return sum(load[t]+model.qin[t])>=0
model.pos_Dem = Constraint(model.T,rule=demand_positive)

In [ ]:
# go for preprocessing
instance=model.create()

In [ ]:
# call glpk solver 
opt = SolverFactory('cplex')
results=opt.solve(instance)

In [ ]:
results.write()


In [ ]:
from pyomo.environ import value

In [ ]:
# create list
soc = []
charge = []
discharge = []
BinC = []
BinD = []
for i in model.T:
    soc.append(value(instance.z[i]))
    charge.append(value(instance.qin[i]))
    discharge.append(value(instance.qout[i]))
    BinC.append(value(instance.Bc[i]))
    BinD.append(value(instance.Bd[i]))

In [ ]:
import plotly.graph_objects as go
import numpy as np

x = time

fig1 = go.Figure(data=go.Scatter(x=x, y=soc))
fig1.update_layout(
    title=go.layout.Title(
        text="PV+BESS",
        xref="paper",
        x=0
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text="Time",
            font=dict(
                family="Courier New, monospace",
                size=18,
                color="#7f7f7f"
            )
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text="SoC (KWh)",
            font=dict(
                family="Courier New, monospace",
                size=18,
                color="#7f7f7f"
            )
        )
    )
)

fig1.show()

fig2 = go.Figure(data=go.Scatter(x=x, y=charge))

fig2.update_layout(
    title=go.layout.Title(
        text="PV+BESS",
        xref="paper",
        x=0
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text="Time",
            font=dict(
                family="Courier New, monospace",
                size=18,
                color="#7f7f7f"
            )
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text="Rate of charging (KW)",
            font=dict(
                family="Courier New, monospace",
                size=18,
                color="#7f7f7f"
            )
        )
    )
)

fig2.show()

fig3 = go.Figure(data=go.Scatter(x=x, y=discharge))
fig3.update_layout(
    title=go.layout.Title(
        text="PV+BESS",
        xref="paper",
        x=0
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text="Time",
            font=dict(
                family="Courier New, monospace",
                size=18,
                color="#7f7f7f"
            )
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text="Rate of discharging (KW)",
            font=dict(
                family="Courier New, monospace",
                size=18,
                color="#7f7f7f"
            )
        )
    )
)

fig3.show()

fig4 = go.Figure(data=go.Scatter(x=x, y=solar))
fig4.update_layout(
    title=go.layout.Title(
        text="PV+BESS",
        xref="paper",
        x=0
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text="Time",
            font=dict(
                family="Courier New, monospace",
                size=18,
                color="#7f7f7f"
            )
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text="Solar Generation (kW)",
            font=dict(
                family="Courier New, monospace",
                size=18,
                color="#7f7f7f"
            )
        )
    )
)

fig4.show()



In [ ]:
import xlsxwriter
results = pd.DataFrame(list(zip(time, soc, charge, discharge)), 
               columns =['Time stamp', 'State of Charge', 'Rate of Charging', 'Rate of Discharge'])

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('problem2.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
results.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()